In [4]:
!pip install pymystem3
!pip install xgboost

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 16.6MB 1.6MB/s eta 0:00:01
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import re
import math
import numpy as np
import pandas as pd
import string
from tqdm import tqdm
from pymystem3 import Mystem
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

/home/user/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


**Цель конкурса** состоит в том чтобы научится по описанию вакансии понимать её категорию.

**Данные содержат** около 15000 вакансий (трейн + тест), которые разбиты на 30 категорий профессий. Каждая вакансия содержит: 

* title — заголовок вакансии
* responsibility — описание обязанностей
* requirement — описание требований к кандидату
* condition — условия, предлагаемые работодателями
* location — название компании
* schedule — график работы

**Метрикой качества будет служить** Micro Average F1 score

In [2]:
vacancies_train = pd.read_csv('vacancies_train.csv')
vacancies_test = pd.read_csv('vacancies_test.csv')

In [3]:
vacancies_train.head()

,category,condition,location,requirement,responsibility,schedule,title,id
0,1,"График работы 5/2, с 9 до 18;\nОформление в со...",ЭКСЕКО Пи Эм Рус,Высшее образование;\nОпыт работы в бухгалтерии...,Осуществление документооборота между подраздел...,полный рабочий день,Делопроизводитель (бухгалтер на первичную доку...,7231
1,1,Мы гарантируем:\n- Оформление по ТК РФ\n- Карь...,ПрезиДЕНТ,У вас в обязательном порядке должно быть:\n- Д...,- Проведение первичных консультаций пациентов\...,полный рабочий день,Заместитель главного врача / Главный врач в Мо...,5814
2,23,"Работа и проживание в МО, Ногинский район. д. ...",АВТОБАН,—Знание технологии укладки асфальтобетона• Не ...,"—Техническое обслуживание машины, проверка исп...",полный рабочий день,Помощник машиниста укладчика асфальтобетона в ...,1245
3,23,- Работа в крупном стабильном холдинге\n- Комф...,Желдорпроект,- Знание и умение работать с нормативными доку...,- Сбор и оценка исходных данных для проектиров...,полный рабочий день,Инженер-технолог / Инженер-проектировщик разде...,4053
4,1,"ЦАО, график работы 5/2, с 9-00 (ненормированны...",NaN,"Образование высшее, подробное резюме.\nОпыт ра...","1. Планирование рабочего дня руководителя, док...",полный рабочий день,Помощник руководителя в Москве,12673


In [4]:
vacancies_train.schedule.unique()

array(['полный рабочий день', 'сменный график работы',
       'неполный рабочий день', 'работа вахтовым методом',
       'частичная занятость / совместительство',
       'временная работа / freelance'], dtype=object)

**Предобработка данных**

Этап 1. Отчищение от "мусора":
    * Удаляем из заголовков фразу "в Москве" (выкачивали только московские вакансии)
    * Удаляем из заголовков информацию в скобках (там как правило указана станция метро или адресс)
    
Этап 2. Лемматизация

In [5]:
lemmatizer = Mystem()

table = str.maketrans({key: ' ' for key in string.punctuation})

def preprocess(s):
    return s.strip().translate(table).casefold()

def filter_word(word):
    return len(word) > 2 and word[0] not in '0123456789'

def clean_sentence(sentence):
    if type(sentence) is not str and math.isnan(sentence):
        return ''
    cleared = re.sub(r'\s+', ' ', re.sub(r'\([^\(\)]*\)', '', re.sub('в Москве', '', sentence)))
    return cleared
    # здесь также пробовал удалять все знаки пунктуации и приводить к lowercase, а также удалять слова, начинающиеся с чисел,
    # скор после этого ухудшился
    # cleared = preprocess(cleared)
    # return ' '.join(filter(filter_word, cleared.split()))

def lemmatize(field):
    clean_title = lambda x: re.sub(r'\s+', ' ', re.sub(r'\([^\(\)]*\)', '', re.sub('в Москве', '', x))).strip()

    tqdm.pandas()
    vacancies_train[f'{field}_lemma'] = vacancies_train[field].apply(clean_sentence).progress_apply(lambda x: ''.join(lemmatizer.lemmatize(x)))
    vacancies_test[f'{field}_lemma'] = vacancies_test[field].apply(clean_sentence).progress_apply(lambda x: ''.join(lemmatizer.lemmatize(x)))

In [6]:
lemmatize('title')
lemmatize('responsibility')
lemmatize('requirement')
lemmatize('condition')

100%|██████████| 4248/4248 [00:05<00:00, 843.49it/s]


In [7]:
vacancies_train.head()

,category,condition,location,requirement,responsibility,schedule,title,id,title_lemma,responsibility_lemma,requirement_lemma,condition_lemma
0,1,"График работы 5/2, с 9 до 18;\nОформление в со...",ЭКСЕКО Пи Эм Рус,Высшее образование;\nОпыт работы в бухгалтерии...,Осуществление документооборота между подраздел...,полный рабочий день,Делопроизводитель (бухгалтер на первичную доку...,7231,делопроизводитель \n,осуществление документооборот между подразделе...,высокий образование;\попыт работа в бухгалтери...,"график работа 5/2, с 9 до 18;\поформление в со..."
1,1,Мы гарантируем:\n- Оформление по ТК РФ\n- Карь...,ПрезиДЕНТ,У вас в обязательном порядке должно быть:\n- Д...,- Проведение первичных консультаций пациентов\...,полный рабочий день,Заместитель главного врача / Главный врач в Мо...,5814,заместитель главный врач / главный врач \n,- проведение первичный консультация пациент\n-...,у вы в обязательный порядок должно быть:\n- де...,мы гарантировать:\n- оформление по тк рф\n- ка...
2,23,"Работа и проживание в МО, Ногинский район. д. ...",АВТОБАН,—Знание технологии укладки асфальтобетона• Не ...,"—Техническое обслуживание машины, проверка исп...",полный рабочий день,Помощник машиниста укладчика асфальтобетона в ...,1245,помощник машинист укладчик асфальтобетон \n,"—технический обслуживание машина, проверка исп...",—знание технология укладка асфальтобетон• не и...,"работа и проживание в мо, ногинский район. д. ..."
3,23,- Работа в крупном стабильном холдинге\n- Комф...,Желдорпроект,- Знание и умение работать с нормативными доку...,- Сбор и оценка исходных данных для проектиров...,полный рабочий день,Инженер-технолог / Инженер-проектировщик разде...,4053,инженер-технолог / инженер-проектировщик разде...,- сбор и оценка исходный данные для проектиров...,- знание и умение работать с нормативный докум...,- работа в крупный стабильный холдинг\n- комфо...
4,1,"ЦАО, график работы 5/2, с 9-00 (ненормированны...",NaN,"Образование высшее, подробное резюме.\nОпыт ра...","1. Планирование рабочего дня руководителя, док...",полный рабочий день,Помощник руководителя в Москве,12673,помощник руководитель \n,"1. планирование рабочий день руководитель, док...","образование высокий, подробный резюме.\попыт р...","цао, график работа 5/2, с 9-00 ,\писпытательны..."


**Базовое решение:** Заголовки вакансий + TfidfVectorizer + LogisticRegression

In [8]:
def add_tfidf(train, test, field):
    vectorizer = TfidfVectorizer()
    # попробовал также vectorizer с параметрами `min_df=10, max_df=1000`, скор ухудшился
    # vectorizer = TfidfVectorizer(min_df=10, max_df=1000)
    vectorizer.fit(train[field])
    train_tfidf = vectorizer.transform(train[field])
    test_tfidf = vectorizer.transform(test[field])
    df_train_tfidf = pd.DataFrame(train_tfidf.toarray(), columns=vectorizer.get_feature_names()).add_prefix(f'{field}_tfidf_')
    df_test_tfidf = pd.DataFrame(test_tfidf.toarray(), columns=vectorizer.get_feature_names()).add_prefix(f'{field}_tfidf_')
    return pd.concat([train, df_train_tfidf], axis=1), pd.concat([test, df_test_tfidf], axis=1)

def add_onehot(train, test, field):
    label_encoder = LabelEncoder()
    label_encoder.fit(train[field])
    
    train_label = label_encoder.transform(train[field]).reshape(len(train), 1)
    test_label = label_encoder.transform(test[field]).reshape(len(test), 1)
    
    onehot_encoder = OneHotEncoder(sparse=False)
    onehot_encoder.fit(train_label)
    
    train_onehot = onehot_encoder.transform(train_label)
    test_onehot = onehot_encoder.transform(test_label)
    
    labels = train[field].sort_values().unique()
    df_train_onehot = pd.DataFrame(train_onehot, columns=labels).add_prefix(f'{field}_onehot_')
    df_test_onehot = pd.DataFrame(test_onehot, columns=labels).add_prefix(f'{field}_onehot_')
    
    return pd.concat([train, df_train_onehot], axis=1), pd.concat([test, df_test_onehot], axis=1)

In [9]:
vacancies_train, vacancies_test = add_tfidf(vacancies_train, vacancies_test, 'title_lemma')
vacancies_train, vacancies_test = add_tfidf(vacancies_train, vacancies_test, 'responsibility_lemma')
vacancies_train, vacancies_test = add_tfidf(vacancies_train, vacancies_test, 'requirement_lemma')
# скор ухудшается, если добавить признаки tfidf для поля `condition`
# vacancies_train, vacancies_test = add_tfidf(vacancies_train, vacancies_test, 'condition_lemma')

In [10]:
vacancies_train, vacancies_test = add_onehot(vacancies_train, vacancies_test, 'schedule')

In [11]:
features = [x for x in vacancies_train.columns if '_tfidf_' in x or '_onehot_' in x]
features[:10], len(features)

(['title_lemma_tfidf_01',
  'title_lemma_tfidf_04пм',
  'title_lemma_tfidf_05',
  'title_lemma_tfidf_09',
  'title_lemma_tfidf_11',
  'title_lemma_tfidf_14',
  'title_lemma_tfidf_19',
  'title_lemma_tfidf_1c',
  'title_lemma_tfidf_1с',
  'title_lemma_tfidf_1сут'],
 30469)

In [207]:
f1_scorer = make_scorer(f1_score, average='micro')
cross_val_score(LogisticRegression(), vacancies_train[features], vacancies_train.category, scoring=f1_scorer, cv=5, n_jobs=3).mean()

0.6188652381875535

Подберем параметр `C`

In [15]:
for c in np.linspace(1.25, 1.45, 20):
    f1_scorer = make_scorer(f1_score, average='micro')
    score = cross_val_score(LogisticRegression(C=c), vacancies_train[features], vacancies_train.category, scoring=f1_scorer, cv=5, n_jobs=3).mean()
    print(c, score)

1.25 0.6194732497067708
1.2605263157894737 0.6194726393696435
1.2710526315789474 0.6195732966908638
1.2815789473684212 0.6195738027485003
1.2921052631578946 0.6193731002898952
1.3026315789473684 0.6191713838693088
1.313157894736842 0.6190704223028789
1.3236842105263158 0.6186668318103503
1.3342105263157895 0.6189703201002125
1.3447368421052632 0.6188689549089287
1.3552631578947367 0.6189688423949576
1.3657894736842104 0.6190691936242602
1.3763157894736842 0.618867319552318
1.3868421052631579 0.6187666088592327
1.3973684210526316 0.618665392108511
1.4078947368421053 0.618665392108511
1.418421052631579 0.6191690277384242
1.4289473684210525 0.6189667500416282
1.4394736842105262 0.6194704298326126
1.45 0.6194701237406947


Попробуем использовать RandomForest:

In [40]:
f1_scorer = make_scorer(f1_score, average='micro')
clf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
cross_val_score(clf, vacancies_train[features], vacancies_train.category, scoring=f1_scorer, cv=5).mean()

0.5798045212563485

...а также XGBoost:

In [36]:
def cross_val_xgb(params, X, y, cv=5):
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    for train_index, test_index in tqdm(kf.split(X)):
        train_dmatrix = xgb.DMatrix(X.iloc[train_index], y[train_index])
        test_dmatrix = xgb.DMatrix(X.iloc[test_index])
        
        clf = xgb.train(params, train_dmatrix, 50)
        y_pred = clf.predict(test_dmatrix)
        
        scores.append(f1_score(y[test_index], y_pred, average='micro'))
        break
    return np.mean(scores)

In [37]:
params = {'objective': 'multi:softmax', 'silent': True, 'num_class': 30}
cross_val_xgb(params, vacancies_train[features], vacancies_train.category)

0it [00:00, ?it/s]

0.56732223903177

В итоге лучшим оказалась логистическая регрессия с `C=1.139795918367347`

In [14]:
classifier = LogisticRegression(C=1.139795918367347)
classifier.fit(vacancies_train[features], vacancies_train.category)

vacancies_test['category'] = classifier.predict(vacancies_test[features])
vacancies_test[['id', 'category']].to_csv('baseline.csv', index=False)

In [17]:
classifier = LogisticRegression(C=1.2815789473684212)
classifier.fit(vacancies_train[features], vacancies_train.category)

vacancies_test['category'] = classifier.predict(vacancies_test[features])
vacancies_test[['id', 'category']].to_csv('baseline.csv', index=False)